# Spark Configuration

In [1]:
import findspark
findspark.init("C:\\Users\\Owner\\scoop\\apps\\spark\\current",)

In [2]:
from pyspark import SparkContext, SparkConf
conf = SparkConf()# .setAppName("app-name-of-your-choice").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

In [4]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [5]:
spark

# Texting Parsing

In [6]:
import pandas as pd 
import json

Input data file

In [7]:
raw_lines = sc.textFile("accor_data.txt")

In [8]:
records = raw_lines.filter(lambda x: x != '').map(lambda x: json.loads(x))

In [10]:
records.take(2)

[{'created_at': 'Sat Dec 08 03:46:45 +0000 2018',
  'id': 1071249696654450688,
  'id_str': '1071249696654450688',
  'text': 'BOYS: FAIRMONT SENIOR POLAR BEARS 77, WYOMING EAST WARRIORS 58 (FAIRMONT SENIOR TOURNAMENT). More final scores: https://t.co/h8yMyFuPjr',
  'source': '<a href="http://www.basketballnight.com" rel="nofollow">Basketball Night</a>',
  'truncated': False,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 865717002,
   'id_str': '865717002',
   'name': 'Basketball Night',
   'screen_name': 'Hoops_Roundup',
   'location': 'West Virginia',
   'url': 'https://basketballnight.com',
   'description': '3 hour live radio show recapping WV high school basketball every Friday night 9pm-midnight across West Virginia. WATCH OR LISTEN: https://t.co/7q0ilTGv7s',
   'translator_type': 'none',
   'protected': False,
   'verified': False,
   'follo

In [11]:
records.count()

1887

text exploration

In [12]:
word_counts = (
    records
    .map(lambda x: x['text'])     # extract the text
    .flatMap(lambda x: x.split())  # split on whitespace and
    # convert each word into a separate record.
    .filter(lambda x: len(x) > 3)  # keep words that have 3 or more letters
    .map(lambda x: (x, 1))           # Count each word
    .reduceByKey(lambda a, b: a + b)  # Sum the counts
)

In [13]:
word_counts.map(lambda x: (x[1], x[0])).top(10)

[(2124, 'MERCI'),
 (534, 'Bercy'),
 (531, 'replay.'),
 (531, '4ème'),
 (530, 'https://t.co/bQN3hcWNGJ'),
 (530, '@Orel_san:'),
 (241, 'Fairmont'),
 (150, 'Ibis'),
 (127, 'Sofitel'),
 (119, 'AccorHotels')]

In [14]:
lang_counts = (
    records
    .map(lambda x: x['lang'])     # extract the text
    #.flatMap(lambda x: x.split())  # split on whitespace and
    # convert each word into a separate record.
    #.filter(lambda x: len(x) > 3)  # keep words that have 3 or more letters
    .map(lambda x: (x, 1))           # Count each word
    .reduceByKey(lambda a, b: a + b)  # Sum the counts
)

In [15]:
lang_counts.map(lambda x: (x[1], x[0])).top(10)

[(714, 'en'),
 (713, 'fr'),
 (177, 'ja'),
 (58, 'und'),
 (36, 'in'),
 (28, 'es'),
 (27, 'pt'),
 (23, 'tl'),
 (21, 'de'),
 (20, 'th')]

# Spark DataFrames

In [16]:
type(records)

pyspark.rdd.PipelinedRDD

In [17]:
df = records.map(lambda x: [x['text']]).toDF()

In [18]:
df.show()

+--------------------+
|                  _1|
+--------------------+
|BOYS: FAIRMONT SE...|
|3月〜！！

受験受かってはぴはぴ...|
|RT @SunDevilWBB: ...|
|RT @DPrez304: Div...|
|愛媛のIBISっていう上級大尉にレ...|
|Karthik kickstart...|
|I just entered th...|
|Roadblock at the ...|
|RT @NRsports_NC: ...|
|I just entered th...|
|First drawing on ...|
|Mala idea venir a...|
|@ibisiro Ibis noO...|
|Fairmont Jakarta:...|
|RT @SunDevilWBB: ...|
|Le Club AccorHote...|
|RT @JoshMLowe: 17...|
|#NoTePases Te qui...|
|同じibisで塗ったっていうのが信...|
|@beltrandelrio @F...|
+--------------------+
only showing top 20 rows



In [19]:
pandas_df = df.toPandas()
pandas_df.head()

,_1
0,"BOYS: FAIRMONT SENIOR POLAR BEARS 77, WYOMING ..."
1,3月〜！！\n\n受験受かってはぴはぴになりながら従姉妹の家で遊んだのが1番楽しかった…\n...
2,"RT @SunDevilWBB: Hey @NkealHarry15,\n\nWe’re b..."
3,RT @DPrez304: Division 1 schools are sleeping ...
4,愛媛のIBISっていう上級大尉にレクスで覚醒使わず2落ちという捨てゲーされた😭
